In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 3.6 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=22feac5de988c51482ed2e1464d1691abfea148f7c483b1dfee3bc79ad2ffa6e
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.5 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK   dataframe_csv_EU
 a0529-a0611_US   dataframe_csv_UK
 a0529.csv	  dataframe_result_csv
 a0530.csv	  dataframe_result_csv_EU
 a0531.csv	  dataframe_result_csv_UK
 a0601.csv	  Github
 a0602.csv	  heatmap529_UK.ipynb
 a0603.csv	  heatmap530.ipynb
 a0604.csv	  Heat_Map_Online_language.ipynb
 a0605.csv	  __MACOSX
 a0606.csv	  non_violant1.csv
 a0607.csv	  non_violant2.csv
 csv		  UK
 csv.zip	  violant.csv
 dataframe_csv	 '新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [8]:
ct_path_531 = 'UK/06_10.csv'

In [9]:
ct = pd.read_csv(ct_path_531)

In [10]:
ct.shape[1]

47

In [11]:
#columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [12]:
#len(columns1)
len(columns2)

47

In [13]:
ct.columns=columns2

In [14]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,343035347444_10158602928237445,Facebook,2020-06-10 23:59:50,2021-09-01 22:12:28,native_video,NaN,NaN,NaN,Texas Southern University offers George Floyd'...,1.0,https://www.facebook.com/News4SA/videos/264978...,https://www.facebook.com/343035347444/posts/10...,684387,-2.907407,2.0,20,3,6,21,1,2,1,0,0,7,46,27,17,12,20,10,16,9,0,4,25427,News 4 San Antonio,News4SA,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"San Antonio's first TV station, signed on Dec....",True,2010-03-03 18:08:56,en,0,25427|10158602928237445
1,475662029147465_3125366800843628,Facebook,2020-06-10 23:59:48,2021-06-15 21:56:11,link,Minneapolis police slashed tires of cars parke...,nypost.com,Two law enforcement agencies in Minneapolis ha...,Are our tax dollars paying for policemen to sl...,1.0,http://nypost.com/2020/06/09/police-slashed-ti...,https://www.facebook.com/groups/47566202914746...,24874,-1.916667,1.0,3,1,1,0,0,1,0,6,0,0,3,2,3,3,2,3,4,3,0,1,7641227,ABC7 Eyewitness Viewers,News4SA,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,"San Antonio's first TV station, signed on Dec....",False,2010-03-03 18:08:56,en,0,7641227|3125366800843628
2,1706330679634904_2660712290863400,Facebook,2020-06-10 23:59:43,2021-06-29 06:32:24,link,H.E.R. Debuts New Song 'I Can't Breathe' In Ho...,real931.iheart.com,The singer unveiled the new track during her i...,The singer unveiled the new track during her i...,1.0,http://ihe.art/gcReOMl,https://www.facebook.com/1706330679634904/post...,4891,-20.000000,1.0,0,0,0,0,0,0,0,0,0,0,1,2,1,2,1,1,1,1,0,2,1610725,Real 93.1,Real931FM,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The Ville's Hip Hop N' R&B,True,2016-04-28 18:11:26,en,0,1610725|2660712290863400
3,10735877375_10158594422127376,Facebook,2020-06-10 23:59:43,2021-07-14 00:51:12,status,H.E.R. Debuts New Song 'I Can't Breathe' In Ho...,real931.iheart.com,The singer unveiled the new track during her i...,Copied from a friend... ~~~~~~~~~~~~~~ I just ...,1.0,http://ihe.art/gcReOMl,https://www.facebook.com/groups/10735877375/pe...,2439,1.526316,1.0,6,10,2,0,1,0,4,6,0,0,4,2,3,2,1,1,1,4,0,0,11274230,Reject the U.N.,Real931FM,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,The Ville's Hip Hop N' R&B,False,2016-04-28 18:11:26,en,0,11274230|10158594422127376
4,949363698484616_3060189834068648,Facebook,2020-06-10 23:59:42,2021-06-07 04:46:55,photo,H.E.R. Debuts New Song 'I Can't Breathe' In Ho...,real931.iheart.com,The singer unveiled the new track during her i...,I've noticed many companies have expressed the...,1.0,https://www.facebook.com/Teliariversmediacoach...,https://www.facebook.com/949363698484616/posts...,1204,-26.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,0,5,0,0,0,0,6056141,Telia Rivers - Producer/Media Coach,Teliariversmediacoach,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,I provide tips and media training to individua...,False,2015-11-02 03:10:47,en,0,6056141|3060189834068648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21301,823195567820794_1828275643979443,Facebook,2020-06-10 15:30:11,2021-08-18 06:39:48,photo,25 Questions from a South

In [15]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [16]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [17]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [19]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [20]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [21]:
ct['NER-msg'] = ct['message'].apply(ner)

In [22]:
ct['NER-desc'] = ct['description'].apply(ner)

In [23]:
ct['NER-title'] = ct['title'].apply(ner)

In [24]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{},{}
1,{},{'Minneapolis': 1}
2,{},{}
3,{},{}
4,{},{}


In [25]:
ct.to_csv('a0610.csv')

In [26]:
ct=pd.read_csv('a0610.csv',lineterminator="\n")

In [27]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [28]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [29]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
1,{},{'Minneapolis': 1},{'Minneapolis': 1}
6,"{'Franklinville': 1, 'New Jersey': 1}","{'Franklinville': 1, 'New Jersey': 1}",{}
7,{'Houston': 1},"{'Franklinville': 1, 'New Jersey': 1}",{'Houston': 1}
8,{},{'Minneapolis': 1},{'Houston': 1}
9,{},"{'RHINE': 1, 'Ohio': 1, 'Minneapolis': 1, 'Cin...",{}


In [30]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [31]:
ct_ner['NER-msg']

1                                 []
6        [Franklinville, New Jersey]
7                          [Houston]
8                                 []
9                                 []
                    ...             
21300                             []
21301                             []
21303      [New Jersey, Minneapolis]
21304                           [UK]
21305                           [UK]
Name: NER-msg, Length: 13689, dtype: object

In [32]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
ct_ner['NER']

1                               [Minneapolis, Minneapolis]
6        [Franklinville, New Jersey, Franklinville, New...
7            [Houston, Franklinville, New Jersey, Houston]
8                                   [Minneapolis, Houston]
9                   [Minneapolis, RHINE, Cincinnati, Ohio]
                               ...                        
21300                                  [South Minneapolis]
21301                                  [South Minneapolis]
21303                [New Jersey, Minneapolis, New Jersey]
21304                                             [UK, UK]
21305                                             [UK, UK]
Name: NER, Length: 13689, dtype: object

In [34]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
ner_lst = ct_ner['NER'].values

In [37]:
ner_lst

array([list(['minneapolis']), list(['franklinville', 'new jersey']),
       list(['franklinville', 'houston', 'new jersey']), ...,
       list(['new jersey', 'minneapolis']), list(['uk']), list(['uk'])],
      dtype=object)

In [38]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [39]:
list

['minneapolis',
 'franklinville',
 'new jersey',
 'franklinville',
 'houston',
 'new jersey',
 'minneapolis',
 'houston',
 'minneapolis',
 'rhine',
 'cincinnati',
 'ohio',
 'fallon',
 'nevada',
 'tereorist',
 'america',
 'yorubas',
 'nigeria',
 'katsina',
 'nigeria',
 'buratai',
 'china',
 'nevada',
 'jasper county',
 'jasper county',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'los angeles',
 'china',
 'the kingdom of heaven',
 'the united states',
 'minneapolis',
 'nyc',
 'new york city',
 'assisi',
 'new york city',
 'metuchen',
 'chicago',
 'houston',
 'chicago',
 'minneapolis',
 'rhine',
 'cincinnati',
 'ohio',
 'houston',
 'minneapolis',
 'minneapolis',
 'new jersey',
 'minneapolis',
 'metuchen',
 'minneapolis',
 'metuchen',
 'arizona',
 'washington',
 'minneapolis',
 'the united states',
 'minnesota',
 'minneapolis',
 'nyc',
 'new york city',
 'christiansburg',
 'the montgomery county sheriff’s office',
 'new jersey',
 'franklinville',
 'us',
 'minneapolis',
 'san francisco

In [40]:
c = collections.Counter()

In [41]:
dict_a=collections.Counter(list).most_common()

In [42]:
dict_a

[('minneapolis', 4077),
 ('us', 1689),
 ('america', 1587),
 ('houston', 1537),
 ('u.s.', 872),
 ('the united states', 844),
 ('texas', 682),
 ('washington', 605),
 ('new york', 553),
 ('minnesota', 548),
 ('new jersey', 465),
 ('california', 311),
 ('uk', 282),
 ('nj', 271),
 ('buffalo', 247),
 ('chicago', 239),
 ('florida', 227),
 ('los angeles', 208),
 ('usa', 203),
 ('london', 194),
 ('d.c.', 181),
 ('new york city', 178),
 ('australia', 175),
 ('south jersey', 169),
 ('seattle', 162),
 ('boston', 160),
 ('canada', 159),
 ('virginia', 142),
 ('nigeria', 131),
 ('oakland', 126),
 ('alabama', 123),
 ('ghana', 122),
 ('china', 121),
 ('atlanta', 121),
 ('maryland', 117),
 ('germany', 116),
 ('georgia', 113),
 ('nyc', 110),
 ('paris', 110),
 ('france', 110),
 ('north carolina', 106),
 ('franklinville', 103),
 ('n.j.', 102),
 ('philadelphia', 101),
 ('new zealand', 100),
 ('britain', 97),
 ('israel', 95),
 ('ireland', 94),
 ('san francisco', 93),
 ('the united states of america', 93),
 (

In [43]:
dataframe531=pd.DataFrame(dict_a)

In [44]:
len(dict_a)

2834

In [45]:
dataframe531.to_csv('dataframe0610_UK.csv')